In [ ]:
%%capture
!pip install --quiet evaluate
!pip install -U sentence-transformers
!pip install --quiet underthesea
!pip install accelerate

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer ,AutoModelForSequenceClassification, TrainingArguments, Trainer , AutoModelForQuestionAnswering , AdamW , get_scheduler , DataCollatorWithPadding
from datasets import load_dataset , concatenate_datasets
#from accelerate import Accelerator
import evaluate
from torch.utils.data import DataLoader
from accelerate import Accelerator
#from transformers import DataCollatorWithPadding
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit , train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from datasets import Dataset,DatasetDict
import collections
import evaluate
from huggingface_hub import notebook_login
# from underthesea import sent_tokenize ,  text_normalize , word_tokenize
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
import re
import torch.nn as nn
import pandas as pd
import seaborn as sns
from underthesea import word_tokenize
SEED = 1337

In [ ]:
train_pandas = pd.read_json ("/kaggle/input/fold-1345/TRAIN CONTEXT SPLIT 3")
val_pandas = pd.read_json ("/kaggle/input/fold-1345/Validation CONTEXT SPLIT 3")
train_pandas = train_pandas.drop ("__index_level_0__" , axis = "columns")
val_pandas = val_pandas.drop ("__index_level_0__" , axis = "columns")
ds_dict = {'train' : Dataset.from_pandas(train_pandas),
           'test' : Dataset.from_pandas(val_pandas)  }
ds = DatasetDict(ds_dict)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
checkpoint = "VietAI/vit5-base"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint , num_labels = 3)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# accuracy = evaluate.load("accuracy")
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
cls_tok = "<s>"
sep_tok = "</s>"
def fix_input (sample) :
    return {"input" : cls_tok + ' ' + sample['claim'] + ' ' + sep_tok +  ' ' +sample['new_context'] + ' ' + sep_tok  }
data_fix_input = ds.map (fix_input )

In [ ]:
data_fix_input['train'][0]

In [ ]:
def tokenize (sample) :
    return tokenizer(sample ["input"] , truncation= True , max_length = 768)
data_tokenize = data_fix_input.map (tokenize , batched = True )
final_data = data_tokenize.remove_columns (['context', 'claim', 'verdict', 'evidence', 'domain', '__index_level_0__', 'new_context','input'])
final_data

In [ ]:
epochs = 3
# device = torch.device ("cuda") if torch.cuda.is_available() else torch.device ("cpu" )
# model.to(device)
#Dataloader
train_dataloader = DataLoader (final_data["train"], batch_size=2, shuffle=True ,collate_fn = data_collator )
val_dataloader = DataLoader (final_data["test"], batch_size=8, shuffle=True , collate_fn = data_collator)
# test_dataloader = DataLoader (split_dataset["test"] , batch_size=16, shuffle=True , collate_fn = data_collator )
#Scheduler
step_to_train = epochs * len (train_dataloader)
optimizer = torch.optim.AdamW (model.parameters () , lr = 2e-5)
scheduler = get_scheduler (name = "cosine" , 
                           optimizer = optimizer ,
                           num_warmup_steps=0,
                           num_training_steps = step_to_train )

#accelerator
accelerator = Accelerator ()
train_dataloader ,val_dataloader ,model , optimizer= accelerator.prepare (train_dataloader  , val_dataloader , model , optimizer)

#optimzier and scheduler
# step_to_train = epochs * len (train_dataloader)
# optimizer = torch.optim.AdamW (model.parameters () , lr = 5e-5)
# scheduler = get_scheduler (name = "linear" , 
#                            optimizer = optimizer ,
#                            num_warmup_steps=0,
#                            num_training_steps = step_to_train
#                           )

#Metric
metric_train_acc = evaluate.load("accuracy")
metric_valid_acc = evaluate.load("accuracy")
metric_train_f1 = evaluate.load("f1")
metric_valid_f1 = evaluate.load("f1")
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return metric.compute(predictions=predictions, references=labels)
#progress_bar = tqdm(range(step_to_train))

     

In [ ]:
from tqdm.auto import tqdm
best_val_acc = 0
i= 0 
model.train ()
for epoch in range (epochs) :
    losses = []
    for batch in tqdm (train_dataloader)  :
        labels = batch.pop ("labels")
        output = model (**batch)
        logits = output.logits
        criterion = nn.CrossEntropyLoss()
        #print (outputs.shape , labels.shape)
        loss = criterion(logits,labels)
        
        accelerator.backward (loss)
        
        optimizer.step ()
        scheduler.step ()
        optimizer.zero_grad ()
        
        losses.append (loss.item())
        predict = torch.argmax (logits, dim = -1)
        metric_train_acc.add_batch (references=labels, predictions = predict)
        metric_train_f1.add_batch (references=labels, predictions = predict)

    print ("Epoch : {} , Loss : {} , Accuracy : {} , F1 : {}".format (epoch + 1 , np.mean( losses) , metric_train_acc.compute ()['accuracy'] , metric_train_f1.compute(average="weighted")['f1'] ) )


    model.eval()
    val_losses = []

    for batch in tqdm (val_dataloader) :
        
        with torch.no_grad () :
            labels = batch.pop ("labels")
            output = model (**batch)
            logits = output.logits
            criterion = nn.CrossEntropyLoss()
            #print (outputs.shape , labels.shape)
            loss = criterion(logits,labels)
            val_losses.append (loss.item())
            
        predict = torch.argmax (output.logits , dim = -1)
        metric_valid_acc.add_batch (references=labels, predictions = predict)
        metric_valid_f1.add_batch (references=labels, predictions = predict )


    val_acc = metric_valid_acc.compute ()['accuracy']
    if val_acc > best_val_acc :
        torch.save(model.state_dict(), "Phobert_v2_best_model_full_data_{}".format (i))
        best_val_acc = val_acc
        i+=1
    print ("Epoch : {}, Val Loss: {} , Validation  ACC Result : {} , Validation F1 Result :{}".format (epoch + 1, np.mean( val_losses) , val_acc , metric_valid_f1.compute (average="weighted")['f1'] ))
                      

In [ ]:
val_acc

In [ ]:
import gc
gc.collect()

In [ ]:
from IPython.display import FileLink
FileLink(r"Phobert_v2_best_model_full_data_1")